# Examine the Evalution Metrics

Examine the resulting model evaluation after the pipeline completes. Download the resulting evaluation.json file from S3 and print the report.

In [1]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r pipeline_name

In [3]:
print(pipeline_name)

BERT-pipeline-1611875862


In [4]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)['PipelineExecutionSummaries']
pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
print(pipeline_execution_status)

while pipeline_execution_status=='Executing':
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)['PipelineExecutionSummaries']
        pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
#        print('Executions for our pipeline...')
#        print(pipeline_execution_status)
    except Exception as e:
        print('Please wait...')
        time.sleep(30)    
    
pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-1611875862/execution/1ckhfo5elhae',
  'PipelineExecutionDisplayName': 'execution-1611876199014',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 1, 28, 23, 23, 18, 683000, tzinfo=tzlocal())}]
CPU times: user 9.31 ms, sys: 3.61 ms, total: 12.9 ms
Wall time: 79.2 ms


# List Pipeline Execution Steps

In [5]:
pipeline_execution_status = executions_response[0]['PipelineExecutionStatus']
print(pipeline_execution_status)

Succeeded


In [6]:
pipeline_execution_arn = executions_response[0]['PipelineExecutionArn']
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:835319576252:pipeline/bert-pipeline-1611875862/execution/1ckhfo5elhae


In [7]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'EndTime': datetime.datetime(2021, 1, 28, 23, 52, 25, 868000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:model-package/bert-reviews-1611876067/1'}},
                             'StartTime': datetime.datetime(2021, 1, 28, 23, 52, 24, 589000, tzinfo=tzlocal()),
                             'StepName': 'RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'EndTime': datetime.datetime(2021, 1, 28, 23, 52, 25, 410000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:model/pipelines-1ckhfo5elhae-createmodel-qmsqebxgnx'}},
                             'StartTime': datetime.datetime(2021, 1, 28, 23, 52, 24, 587000, tzinfo=tzlocal()),
                             'StepName': 'CreateModel',
                             'StepStatus': 'Succeeded'},
                  

# Retrieve Evaluation Metrics

In [8]:
#for execution_step in reversed(execution.list_steps()):
for execution_step in reversed(steps['PipelineExecutionSteps']):
    if execution_step['StepName'] == 'EvaluateModel':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]

describe_evaluation_processing_job_response = sm.describe_processing_job(ProcessingJobName=processing_job_name)

evaluation_metrics_s3_uri = describe_evaluation_processing_job_response['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
evaluation_metrics_s3_uri

's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2021-01-28-23-21-45-903/output/metrics'

In [10]:
import json
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    evaluation_metrics_s3_uri
))

pprint(json.loads(evaluation_json))

{'metrics': {'accuracy': {'value': 0.53}}}


# Download the Trained S3 Model

In [11]:
training_job_arn=None

for execution_step in steps['PipelineExecutionSteps']:
    if execution_step["StepName"] == "Train":
        training_job_arn = execution_step["Metadata"]["TrainingJob"]["Arn"]
        
        print(execution_step)
        break
print(training_job_arn)
        
training_job_name = training_job_arn.split('/')[-1]
print(training_job_name)

{'StepName': 'Train', 'StartTime': datetime.datetime(2021, 1, 28, 23, 34, 4, 656000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 43, 52, 91000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:training-job/pipelines-1ckhfo5elhae-train-dggs3tm60e'}}}
arn:aws:sagemaker:us-east-1:835319576252:training-job/pipelines-1ckhfo5elhae-train-dggs3tm60e
pipelines-1ckhfo5elhae-train-dggs3tm60e


In [12]:
model_tar_s3_uri = sm.describe_training_job(TrainingJobName=training_job_name)['ModelArtifacts']['S3ModelArtifacts']


In [13]:
!aws s3 cp $model_tar_s3_uri ./
    

download: s3://sagemaker-us-east-1-835319576252/pipelines-1ckhfo5elhae-Train-DGgs3tM60E/output/model.tar.gz to ./model.tar.gz


In [14]:
!mkdir -p ./downloaded_model 
!tar -zxvf model.tar.gz -C ./downloaded_model 


code/
code/inference.py
tensorboard/
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00001
tensorflow/saved_model/0/saved_model.pb
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
test_data/
test_data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz


In [15]:
!saved_model_cli show --all --dir ./downloaded_model/tensorflow/saved_model/0/


2021-01-28 23:59:49.768883: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-01-28 23:59:49.768936: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 64)
        name: serving_defau

# List All Artifacts Generated By The Pipeline

In [16]:
processing_job_name=None
training_job_name=None

In [17]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps['PipelineExecutionSteps']):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

{'StepName': 'Processing', 'StartTime': datetime.datetime(2021, 1, 28, 23, 23, 19, 398000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 34, 3, 966000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:processing-job/pipelines-1ckhfo5elhae-processing-hgumk7zqnj'}}}
pipelines-1ckhfo5elhae-processing-hgumk7zqnj


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...ess-scikit-text-to-bert-feature-store.py,Input,DataSet,ContributedTo,artifact
1,s3://...t-1-835319576252/amazon-reviews-pds/tsv/,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...2021-01-28-23-23-03-097/output/bert-test,Output,DataSet,Produced,artifact
4,s3://...1-28-23-23-03-097/output/bert-validation,Output,DataSet,Produced,artifact
5,s3://...021-01-28-23-23-03-097/output/bert-train,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2021, 1, 28, 23, 34, 4, 656000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 43, 52, 91000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:training-job/pipelines-1ckhfo5elhae-train-dggs3tm60e'}}}
pipelines-1ckhfo5elhae-train-dggs3tm60e


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...2021-01-28-23-23-03-097/output/bert-test,Input,DataSet,ContributedTo,artifact
1,s3://...1-28-23-23-03-097/output/bert-validation,Input,DataSet,ContributedTo,artifact
2,s3://...021-01-28-23-23-03-097/output/bert-train,Input,DataSet,ContributedTo,artifact
3,76310...s.com/tensorflow-training:2.3.1-cpu-py37,Input,Image,ContributedTo,artifact
4,s3://...hae-Train-DGgs3tM60E/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'EvaluateModel', 'StartTime': datetime.datetime(2021, 1, 28, 23, 43, 52, 632000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 52, 15, 317000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:processing-job/pipelines-1ckhfo5elhae-evaluatemodel-06smzqwjkv'}}}


...

None

{'StepName': 'CreateModel', 'StartTime': datetime.datetime(2021, 1, 28, 23, 52, 24, 587000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 52, 25, 410000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:model/pipelines-1ckhfo5elhae-createmodel-qmsqebxgnx'}}}


None

{'StepName': 'RegisterModel', 'StartTime': datetime.datetime(2021, 1, 28, 23, 52, 24, 589000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2021, 1, 28, 23, 52, 25, 868000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:835319576252:model-package/bert-reviews-1611876067/1'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...hae-Train-DGgs3tM60E/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,76310...onaws.com/tensorflow-inference:2.3.1-cpu,Input,Image,ContributedTo,artifact
2,bert-reviews-1611876067-1-PendingManualApprova...,Input,Approval,ContributedTo,action
3,BERT-Reviews-1611876067-1611877945-aws-model-p...,Output,ModelGroup,AssociatedWith,context


# Analyze Experiment

In [20]:
%store -r pipeline_experiment_name

In [21]:
from sagemaker.analytics import ExperimentAnalytics

time.sleep(30)

import pandas as pd
pd.set_option("max_colwidth", 500)

experiment_analytics = ExperimentAnalytics(
    experiment_name=pipeline_experiment_name,
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

""


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}